Preliminary Note: Recall that I cannot use any specifically markdown formatting in the notebook, since opening it in PyCharm (a necessary operation to ensure it has saved code appropriately) will permanently delete any cell with special markdown formatting.

Vision Revision:
For my final project, I propose to build a neural network trained to generate poetry in the style of John  Milton. My dataset will be a simple text file containing the entirety of Paradise Lost (about 11,000 lines of unrhyming iambic pentameter (also known as blank verse)).

Background:
This project will use an LSTM (Long short-term memory) trained on the text of Paradise Lost. An LSTM is a specific type of recurrent neural network (RNN) which is structured in order to be able to learn longer term interdependencies between inputs. This LSTM will analyze the text of Paradise Lost, tokenized by letter, and will generate output coorespondingly. My model for this will be Chollet (https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/8.1-text-generation-with-lstm.ipynb) with some cross-referencing of another example (https://github.com/pranjal52/text_generators) which was unfortunately running too slow on the lab computers to be very useful, but did provide some structural guidance.

Implementation:
My implementation is not significantly different from Chollet's at this point. Once it is, I will have more to say on this matter.

Results:
Again, since my implementation is about the same as Chollet's currently, I do not have final results yet. That being said, these preliminary results are very promising, I think, and there is only room for improvement from here with finetuning Chollet's model.

Implications:
Poetry is weird. A conglomeration of semi-coherent, half-connected words and phrases whose structure and interplay somehow amount to far more than their mere parts -- it does not seem like the sort of thing that a NN or AI (at least currently) could convincingly fake. It's too human, too subjective, and toes the line between nonsense and beauty far too closely. But the fact that it is subjective and often so open to interpretation is exactly what makes the potential for NN-generated poetry so great. If someone doesn't know that a poem has been generated by an un-thinking AI (and no AI that I am aware of can currently be described as thinking, in a human sense), they will impute their own interpretations and understandings into the poem and attribute them to the author. In other words, if a neural network can generate convingingly human poetry, it is due in large part to how nebulous and varying poetry is in the first place. But that fact is simultaneously why it would be difficult to train a neural network to generate convincing poetry. The variation and inconsistency that makes poetry such a uniquely human affair also makes it difficult to train a neural network on. This is one of the reasons why I have trouble believing any AI will ever be up to or exceed the capacity of a human beings in their own domains (obviously computers can easily surpass human beings at, say, raw computational power and things of that sort). There is something about us that makes us and our cultural activities distinct from, and greater than, mere mathematics.

In [1]:
import keras
import numpy as np

text = open("projectData.txt").read().lower()
print('Corpus length:', len(text))

Using TensorFlow backend.


Corpus length: 454312


In [2]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
print(chars)
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 151418
Unique characters: 38
['\n', ' ', '!', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Vectorization...


In [3]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
print("Done")

Instructions for updating:
Colocations handled automatically by placer.
Done


In [4]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
print("Done")

Done


In [5]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

print("Done")

Done


In [8]:
import random
import sys

print("epoch")
# Fit the model for 1 epochs on the available training data
model.fit(x, y, batch_size=32, epochs=1)

epoch
Epoch 1/1
151418/151418 [==============================] - 95s 628us/step - loss: 1.8025


In [12]:
# Select a text seed at random
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]
print('--- Generating with seed: "' + generated_text + '"')

for temperature in [0.2, 0.5, 1.0, 1.2]:
    print('------ temperature:', temperature)
    sys.stdout.write(generated_text)

    # We generate 400 characters
    for i in range(400):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

--- Generating with seed: "e dawning light,
thy ransom paid, which man from death redee"
------ temperature: 0.2
e dawning light,
thy ransom paid, which man from death redeet the saw in his the spirit the streath and the seen, and the stread the stread the see to his the streath the streath the see the starr,
the streath and the streath the speake the sund the strain the stroat and starr the see thir angels to the streath the shall the see his to the see to the sunder the see the sund the sees and stread the sund the streath the streath the see the spread with with t
------ temperature: 0.5
 sund the streath the streath the see the spread with with thir they the starr charge strie, the all the signest strouch the sound the pourse to first the truning, and tender to the first shall the sound with as the will doin to streat so world
to the ended there the first leave, and hast the for the rowing, and many there the beast he states
his day in the tound shall in the sumpent in her stread a